# Neural search demo - initial indexing

Code in this notebook shows how to prepare data for indexing in a vector search engine.

It contains the following steps:

* Downloading text data which we want to search
* Initialization of pre-trained text vectorization models (with SentenceTransformer)
* Converting text data into vectors and saving it.

In [1]:
# We will use startup descriptions in this neural search demo
# Data source: https://startups-list.com/
# It contains name, short descrition, logo and location of startups.
!wget https://storage.googleapis.com/generall-shared-data/startups_demo.json

--2025-06-13 17:26:23--  https://storage.googleapis.com/generall-shared-data/startups_demo.json
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.31.207, 142.250.153.207, 142.250.145.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.31.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22205751 (21M) [application/json]
Saving to: ‘startups_demo.json’

startups_demo.json  100%[===================>]  21.18M  16.8MB/s    in 1.3s    

2025-06-13 17:26:24 (16.8 MB/s) - ‘startups_demo.json’ saved [22205751/22205751]



In [2]:
# We use SentenceTransformer pre-trained models to convert our text into vectors.
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 53.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.8 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
from sentence_transformers import SentenceTransformer
import numpy as np
import json
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
# This code will download and create a pre-trained sentence encoder

# all-MiniLM-L6-v2 - is a distilated (lightweight) version of MPNet model.
# It is optimized for the fast inference.
# Full list of available models could be found here https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer('all-MiniLM-L6-v2', device="cuda")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [5]:
df = pd.read_json('./startups_demo.json', lines=True)

In [6]:
# Here we encode all startup descriptions
# We do encoding in batches, as this reduces overhead costs and significantly speeds up the process
vectors = model.encode([
    row.alt + ". " + row.description
    for row in df.itertuples()
], show_progress_bar=True)

Batches:   0%|          | 0/1265 [00:00<?, ?it/s]

In [7]:
# Now we have all our descriptions converted into vectors.
# We have 40474 vectors of 384 dimentions. The output layer of the model has this dimension
vectors.shape

(40474, 384)

In [ ]:
# You can download this saved vectors and continue with rest part of the tutorial.
np.save('vectors.npy', vectors, allow_pickle=False)

In [ ]:
from google.colab import files
files.download('vectors.npy')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Optional part - make a test query

Let's just make sure, that our vectors are correctly converted and make sense.

For this we manually search for a closest vectors of a random sample.

In [8]:
from sklearn.metrics.pairwise import cosine_similarity

In [9]:
# Take a random description as a query
sample_query = df.iloc[12345].description
print(sample_query)

Dental transparency provider
Deenty is a dentistry marketplace. Find your next dentist in an easy, quick and transparent way. Find reviews, sales prices and education about your next dental treatment.


In [10]:
query_vector = model.encode(sample_query)  # Convert query description into a vector.

In [11]:
scores = cosine_similarity([query_vector], vectors)[0]  # Look for the most similar vectors, manually score all vectors
top_scores_ids = np.argsort(scores)[-5:][::-1]  # Select top-5 with vectors the largest scores

In [12]:
# Check if result similar to the query
for top_id in top_scores_ids:
  print(df.iloc[top_id].description)
  print("-----")

Dental transparency provider
Deenty is a dentistry marketplace. Find your next dentist in an easy, quick and transparent way. Find reviews, sales prices and education about your next dental treatment.
-----
Smiley marketplace. We connect patients with dentists
Deenty let dentists create a profile who let them be viewed by new patients. On the other hand patients get to know their dentists, book online, learn about treatments and compare prices.
We charge dentists everytime they treat one of the patients we send them.
-----
Our mission is to make quality dental care affordable for everyone.
-----
Dental management made easy
Dentalink is a web based monthly fee software, for the management of dental clinics and practices. With it, you can manage all the resources within, from appointment, notifications, email marketing to performance reports, like cash flow, quotes uptake rate, and ...
-----
Cure with confidence
Dentists make over half their income from resin-based fillings. However, the